In [1]:
import movingpandas as mpd
import geopandas as gpd
import pandas as pd
from datetime import datetime, timedelta
from shapely.geometry import LineString, Point
from shapely.wkt import loads

In [2]:
hvplot_defaults = {'tiles':'CartoLight', 'frame_height':320, 'frame_width':320, 'cmap':'Viridis', 'colorbar':True}
kwargs = {**hvplot_defaults, 'c':'speed', 'line_width':7, 'clim':(0,20)}

In [3]:
gdf = gpd.read_parquet('output_data/geolife.parquet')

In [4]:
gdf['time'] = pd.to_datetime(gdf['time'])
gdf.crs = 'EPSG:4326'

In [9]:
gdf.head()

,latitude,longitude,altitude,date,date_str,time,trajectory_id,mode,geometry,user_id
0,39.974294,116.399741,492.0,39816.056644,2009-01-03,2009-01-03 01:21:34,20090103012134,unknown,POINT (116.39974 39.97429),135
1,39.974292,116.399592,492.0,39816.056655,2009-01-03,2009-01-03 01:21:35,20090103012134,unknown,POINT (116.39959 39.97429),135
2,39.974309,116.399523,492.0,39816.056667,2009-01-03,2009-01-03 01:21:36,20090103012134,unknown,POINT (116.39952 39.97431),135
3,39.974320,116.399588,492.0,39816.056690,2009-01-03,2009-01-03 01:21:38,20090103012134,unknown,POINT (116.39959 39.97432),135
4,39.974365,116.399730,491.0,39816.056701,2009-01-03,2009-01-03 01:21:39,20090103012134,unknown,POINT (116.39973 39.97436),135


In [10]:
gdf.shape

(24876978, 10)

In [ ]:
def validate_cords(gdf, lon_col = 'longitude', lat_col = 'latitude'):
    valid_lon = (-180 <= gdf[lon_col]) & (gdf[lon_col] <= 180)
    valid_lat = (-90 <= gdf[lat_col]) & (gdf[lat_col] <= 90)
    
    valid_coords = valid_lon & valid_lat
    
    invalid_coords = (~valid_coords).sum()
    if valid_coords == 0:
        print("Wszystkie współrzędne są poprawne.")
    else:
        print(f"Liczba niepoprawnych współrzędnych: {invalid_coords}")
        print("Niepoprawne wiersze:")
        print(gdf[~valid_coords])

    return valid_coords

In [11]:
traj_col = mpd.TrajectoryCollection(gdf,'trajectory_id', t = 'time', x = 'latitude', y = 'longitude')

In [ ]:
traj_col.add_speed(overwrite = True)
traj_col.add_timedelta(overwrite = True)
traj_col.add_direction(overwrite = True)

In [ ]:
traj_col.trajectories[6].hvplot(**kwargs)+traj_col.trajectories[5].hvplot(**kwargs)

In [ ]:
results = traj_col.to_point_gdf()
results.head()

In [ ]:
results.shape

In [70]:
# results.to_parquet('geolife_points.parquet')